# extract speaker embedding

In [1]:
import model
import torch
import torch, sys, os, tqdm, numpy, soundfile, time, pickle
import torchaudio.transforms as T
import torch.nn.functional as F

use_cuda = torch.cuda.is_available() and True
device = torch.device("cuda" if use_cuda else "cpu")

speaker_encoder = model.ECAPA_TDNN(1024, 16000).to(device)

loaded_state = torch.load('./your_model.model')
self_state = speaker_encoder.state_dict()
new_state_dict = {name.replace("speaker_encoder.", ""): param for name, param in loaded_state.items()}
new_state_dict.pop('speaker_loss.weight')
speaker_encoder.load_state_dict(new_state_dict)

<All keys matched successfully>

In [3]:
sampling_rate = 16000
audio_path = './your_audio_file'
audio, sr  = soundfile.read(os.path.join(audio_path))

audio = torch.FloatTensor(audio)
resampler = T.Resample(sr, sampling_rate, dtype=audio.dtype)
audio = resampler(audio)
with torch.no_grad():
    speaker_encoder.eval()
    embedding = speaker_encoder(audio.unsqueeze(0).to(device), False)
    embedding = F.normalize(embedding, p=2, dim=1)
print(embedding.shape)

torch.Size([1, 192])


# cosine similarity between positive speakers

In [4]:
sampling_rate = 16000
audio_path1 = './your_audio_file1'
audio_path2 = './your_audio_file2'

audio1, sr  = soundfile.read(os.path.join(audio_path1))
audio2, sr  = soundfile.read(os.path.join(audio_path2))

audio1 = torch.FloatTensor(audio1)
resampler = T.Resample(sr, sampling_rate, dtype=audio.dtype)
audio1 = resampler(audio1)

audio2 = torch.FloatTensor(audio2)
resampler = T.Resample(sr, sampling_rate, dtype=audio.dtype)
audio2 = resampler(audio2)

with torch.no_grad():
    speaker_encoder.eval()
    embedding_11 = speaker_encoder(audio1.unsqueeze(0).to(device), False)
    embedding_11 = F.normalize(embedding_11, p=2, dim=1)
    
    embedding_21 = speaker_encoder(audio2.unsqueeze(0).to(device), False)
    embedding_21 = F.normalize(embedding_21, p=2, dim=1)

score = F.cosine_similarity(embedding_11.to(device), embedding_21.to(device))

print(score)

tensor([0.7683], device='cuda:0')


# cosine similarity between negative speakers

In [5]:
sampling_rate = 16000
audio_path1 = './your_audio_file1'
audio_path2 = './your_audio_file2'

audio1, sr  = soundfile.read(os.path.join(audio_path1))
audio2, sr  = soundfile.read(os.path.join(audio_path2))

audio1 = torch.FloatTensor(audio1)
resampler = T.Resample(sr, sampling_rate, dtype=audio.dtype)
audio1 = resampler(audio1)

audio2 = torch.FloatTensor(audio2)
resampler = T.Resample(sr, sampling_rate, dtype=audio.dtype)
audio2 = resampler(audio2)

with torch.no_grad():
    speaker_encoder.eval()
    embedding_11 = speaker_encoder(audio1.unsqueeze(0).to(device), False)
    embedding_11 = F.normalize(embedding_11, p=2, dim=1)
    
    embedding_21 = speaker_encoder(audio2.unsqueeze(0).to(device), False)
    embedding_21 = F.normalize(embedding_21, p=2, dim=1)

score = F.cosine_similarity(embedding_11.to(device), embedding_21.to(device))

print(score)

tensor([-0.0212], device='cuda:0')
